# Quick Start: OpenADR 3 Pricing Demo

**Prerequisites:** The VTN must be running in a separate terminal. See `instructions.ipynb` for setup details.

```bash
cd openadr3-vtn-reference-implementation
virtualenv venv && source venv/bin/activate
pip3 install -r requirements.txt
python -m swagger_server
```

## Step 1: Setup & Verify VTN Connection

In [7]:
import requests
import json
from datetime import datetime, timedelta
import isodate


In [8]:
# --- Configuration ---
VTN_BASE_URL = "http://localhost:8080/openadr3/3.0.1"
BL_CLIENT_ID = "bl_client"
BL_CLIENT_SECRET = "1001"
VEN_CLIENT_ID = "ven_client"
VEN_CLIENT_SECRET = "999"
OLIVINE_PRICING_URL = 'https://api.olivineinc.com/i/oe/pricing/signal/paced/etou-dyn'


def get_token(client_id, client_secret):
    """Get an OAuth token from the VTN."""
    resp = requests.post(
        f"{VTN_BASE_URL}/auth/token",
        data={
            "grant_type": "client_credentials",
            "client_id": client_id,
            "client_secret": client_secret,
        },
    )
    resp.raise_for_status()
    return resp.json()["access_token"]


def bl_headers():
    """Return Authorization headers for Business Logic role."""
    return {"Authorization": f"Bearer {get_token(BL_CLIENT_ID, BL_CLIENT_SECRET)}"}


def ven_headers():
    """Return Authorization headers for VEN role."""
    return {"Authorization": f"Bearer {get_token(VEN_CLIENT_ID, VEN_CLIENT_SECRET)}"}


# --- Verify VTN is running ---
resp = requests.get(f"{VTN_BASE_URL}/programs", headers=bl_headers())
resp.raise_for_status()
print(f"VTN is running. Current programs: {resp.json()}")

VTN is running. Current programs: []


## Step 2: Fetch Live Prices from Olivine API

In [9]:
# Fetch live pricing data from the Olivine API
olivine_resp = requests.get(OLIVINE_PRICING_URL, headers={'Accept': 'application/json'})
olivine_resp.raise_for_status()
olivine_data = olivine_resp.json()

# Extract metadata from vtnComment
vtn_comment = olivine_data["eventDescriptor"]["vtnComment"]
metadata = dict(item.split(':', 1) for item in vtn_comment.split(';'))
print("Metadata:")
for key, value in metadata.items():
    print(f"  {key}: {value}")

# Extract start time using isodate
start_time_str = olivine_data["eiActivePeriod"]["properties"]["dtstart"]["datetime"]["value"]
start_time = isodate.parse_datetime(start_time_str)

# Extract price intervals
intervals = olivine_data["eiEventSignals"]["eiEventSignal"][0]["intervals"]

print(f"\nPricing window start: {start_time}")
print(f"Number of hourly intervals: {len(intervals)}")
print(f"\nHourly prices (USD/kWh):")

current_time = start_time
for interval_data in intervals:
    interval_num = int(interval_data["item"]["text"])
    price = interval_data["streamPayloadBase"][0]["item"]["value"]
    duration_str = interval_data["duration"]["duration"]
    
    # Parse duration and calculate end time
    duration = isodate.parse_duration(duration_str)
    end_time = current_time + duration
    
    print(f"  Hour {interval_num:2d}: ${price:.5f} ({current_time:%Y-%m-%d %H:%M} to {end_time:%Y-%m-%d %H:%M})")
    
    current_time = end_time

Metadata:
  BindingPrices: True
  LocalPrice: False
  RetailerLong: Pacific Utility Company
  RateNameLong: E-TOU Dynamic
  DateAnnounced: 2020-01-01
  DateStart: 2020-06-01
  URL: http://www.example.org/PUC/paced-etou-dyn

Pricing window start: 2026-02-18 00:00:00+00:00
Number of hourly intervals: 24

Hourly prices (USD/kWh):
  Hour  0: $0.12052 (2026-02-18 00:00 to 2026-02-18 01:00)
  Hour  1: $0.12227 (2026-02-18 01:00 to 2026-02-18 02:00)
  Hour  2: $0.12213 (2026-02-18 02:00 to 2026-02-18 03:00)
  Hour  3: $0.12132 (2026-02-18 03:00 to 2026-02-18 04:00)
  Hour  4: $0.12127 (2026-02-18 04:00 to 2026-02-18 05:00)
  Hour  5: $0.12046 (2026-02-18 05:00 to 2026-02-18 06:00)
  Hour  6: $0.11859 (2026-02-18 06:00 to 2026-02-18 07:00)
  Hour  7: $0.11692 (2026-02-18 07:00 to 2026-02-18 08:00)
  Hour  8: $0.11888 (2026-02-18 08:00 to 2026-02-18 09:00)
  Hour  9: $0.11710 (2026-02-18 09:00 to 2026-02-18 10:00)
  Hour 10: $0.11624 (2026-02-18 10:00 to 2026-02-18 11:00)
  Hour 11: $0.11664 (2

In [10]:
intervals

[{'duration': {'duration': 'PT1H'},
  'item': {'text': '0'},
  'streamPayloadBase': [{'item': {'value': 0.12051552}}]},
 {'duration': {'duration': 'PT1H'},
  'item': {'text': '1'},
  'streamPayloadBase': [{'item': {'value': 0.12227417}}]},
 {'duration': {'duration': 'PT1H'},
  'item': {'text': '2'},
  'streamPayloadBase': [{'item': {'value': 0.12212558}}]},
 {'duration': {'duration': 'PT1H'},
  'item': {'text': '3'},
  'streamPayloadBase': [{'item': {'value': 0.12132113}}]},
 {'duration': {'duration': 'PT1H'},
  'item': {'text': '4'},
  'streamPayloadBase': [{'item': {'value': 0.12127424}}]},
 {'duration': {'duration': 'PT1H'},
  'item': {'text': '5'},
  'streamPayloadBase': [{'item': {'value': 0.12045975}}]},
 {'duration': {'duration': 'PT1H'},
  'item': {'text': '6'},
  'streamPayloadBase': [{'item': {'value': 0.11859411}}]},
 {'duration': {'duration': 'PT1H'},
  'item': {'text': '7'},
  'streamPayloadBase': [{'item': {'value': 0.11692154}}]},
 {'duration': {'duration': 'PT1H'},
  'i

In [11]:
# Create a pricing program on the VTN
# Note: The VTN reference implementation uses the 3.1.0 schema internally,
# which only accepts: programName, intervalPeriod, programDescriptions,
# payloadDescriptors, attributes, targets.
# Fields like programType, retailerName, country etc. are 3.0.1-only
# and will be rejected. We store them as attributes instead.
program_data = {
    "programName": "etou-dynamic-pricing",
    "payloadDescriptors": [
        {
            "objectType": "EVENT_PAYLOAD_DESCRIPTOR",
            "payloadType": "PRICE",
            "units": "KWH",
            "currency": "USD"
        }
    ]
}

resp = requests.post(
    f"{VTN_BASE_URL}/programs",
    json=program_data,
    headers=bl_headers(),
)
print(f"Status: {resp.status_code}")
print(f"Response: {resp.text}")
resp.raise_for_status()
program = resp.json()
program_id = program["id"]

print(f"\nProgram created successfully!")
print(f"Program ID: {program_id}")
print(f"\nFull response:")
print(json.dumps(program, indent=2))

Status: 201
Response: {
  "createdDateTime": "2026-02-17 16:16:20",
  "id": "0",
  "objectType": "PROGRAM",
  "programName": "etou-dynamic-pricing"
}


Program created successfully!
Program ID: 0

Full response:
{
  "createdDateTime": "2026-02-17 16:16:20",
  "id": "0",
  "objectType": "PROGRAM",
  "programName": "etou-dynamic-pricing"
}


## Step 4: Publish Price Signal as an Event

Convert the Olivine pricing data into an OpenADR 3 event and publish it to the VTN.

In [12]:
# Convert Olivine pricing data to an OpenADR 3 event
event_data = {
    "eventName": f"etou-dyn-{olivine_data['eventDescriptor']['eventID']}",
    "programID": program_id,
    "intervalPeriod": {
        "start": start_time.isoformat(),
        "duration": "PT1H"
    },
    "payloadDescriptors": [
        {
            "objectType": "EVENT_PAYLOAD_DESCRIPTOR",
            "payloadType": "PRICE",
            "units": "KWH",
            "currency": "USD"
        }
    ],
    "intervals": [
        {
            "id": int(interval_data["item"]["text"]),
            "payloads": [{"type": "PRICE", "values": [round(interval_data["streamPayloadBase"][0]["item"]["value"], 5)]}]
        }
        for interval_data in intervals
    ]
}

resp = requests.post(
    f"{VTN_BASE_URL}/events",
    json=event_data,
    headers=bl_headers(),
)
print(f"Status: {resp.status_code}")
print(f"Response: {resp.text}")
resp.raise_for_status()
event = resp.json()

print(f"\nEvent created successfully!")
print(f"Event ID: {event['id']}")
print(f"Intervals published: {len(event['intervals'])}")
print(f"\nFull response:")
print(json.dumps(event, indent=2))

Status: 201
Response: {
  "createdDateTime": "2026-02-17 16:16:26",
  "eventName": "etou-dyn-PUC-paced-etou-dyn-2026-02-17",
  "id": "0",
  "intervalPeriod": {
    "duration": "PT1H",
    "start": "2026-02-18T00:00:00+00:00"
  },
  "intervals": [
    {
      "id": 0,
      "payloads": [
        {
          "type": "PRICE",
          "values": [
            0.12052
          ]
        }
      ]
    },
    {
      "id": 1,
      "payloads": [
        {
          "type": "PRICE",
          "values": [
            0.12227
          ]
        }
      ]
    },
    {
      "id": 2,
      "payloads": [
        {
          "type": "PRICE",
          "values": [
            0.12213
          ]
        }
      ]
    },
    {
      "id": 3,
      "payloads": [
        {
          "type": "PRICE",
          "values": [
            0.12132
          ]
        }
      ]
    },
    {
      "id": 4,
      "payloads": [
        {
          "type": "PRICE",
          "values": [
            0.12127
     

## Step 5: Read Events as a VEN

Authenticate as a VEN client and read the pricing events from the VTN — this is what a water heater controller would do.

In [13]:
# Read all events as a VEN
resp = requests.get(f"{VTN_BASE_URL}/events", headers=ven_headers())
resp.raise_for_status()
events = resp.json()

print(f"Number of events: {len(events)}")
for evt in events:
    print(f"\n--- Event: {evt.get('eventName', evt['id'])} ---")
    print(f"  Program ID: {evt['programID']}")
    print(f"  Start: {evt.get('intervalPeriod', {}).get('start', 'N/A')}")
    print(f"  Intervals: {len(evt['intervals'])}")
    print(f"  Prices (USD/kWh):")
    for interval in evt["intervals"]:
        for payload in interval["payloads"]:
            if payload["type"] == "PRICE":
                print(f"    Hour {interval['id']:2d}: ${payload['values'][0]:.5f}")

Number of events: 1

--- Event: etou-dyn-PUC-paced-etou-dyn-2026-02-17 ---
  Program ID: 0
  Start: 2026-02-18T00:00:00+00:00
  Intervals: 24
  Prices (USD/kWh):
    Hour  0: $0.12052
    Hour  1: $0.12227
    Hour  2: $0.12213
    Hour  3: $0.12132
    Hour  4: $0.12127
    Hour  5: $0.12046
    Hour  6: $0.11859
    Hour  7: $0.11692
    Hour  8: $0.11888
    Hour  9: $0.11710
    Hour 10: $0.11624
    Hour 11: $0.11664
    Hour 12: $0.11659
    Hour 13: $0.12147
    Hour 14: $0.12621
    Hour 15: $0.12647
    Hour 16: $0.11880
    Hour 17: $0.11455
    Hour 18: $0.11120
    Hour 19: $0.10689
    Hour 20: $0.10519
    Hour 21: $0.10300
    Hour 22: $0.10620
    Hour 23: $0.10930


## Step 6: Run Easy Shift Control Algorithm

Use the Easy Shift algorithm to convert the price signals from the VTN into an optimal heat pump water heater operation schedule. The algorithm ranks hours by electricity cost and shifts load to the cheapest hours while respecting thermal storage constraints.

**Parameters below are for a typical residential heat pump water heater (HPWH):**
- 80-gallon tank (~300 kWh thermal capacity)
- 4.5 kW heat pump output
- COP ~3.0 (constant for simplicity)
- Assumed uniform hot water draw profile across the pricing window

In [ ]:
from controls import easy_shift, get_storage, iteration_plot

# --- Extract prices from the VEN event (Step 5) ---
# Use the first event's price intervals
evt = events[0]
prices = []
for interval in sorted(evt["intervals"], key=lambda x: x["id"]):
    for payload in interval["payloads"]:
        if payload["type"] == "PRICE":
            prices.append(payload["values"][0])

horizon = len(prices)
print(f"Horizon: {horizon} hours")
print(f"Prices ($/kWh): {prices}")

# --- HPWH parameters ---
max_hp_output = 4.5      # kW (heat pump max thermal output)
min_hp_output = 0.0      # kW (fully variable speed)
max_storage = 12.0       # kWh (80-gal tank thermal capacity)
min_storage = 1.0        # kWh (keep small reserve)
initial_soc = 6.0        # kWh (start half full)
cop = 3.0                # constant COP for simplicity

# Assume uniform hot water draw across the window
avg_draw = 1.5  # kWh per hour (typical residential draw)
load_per_hour = [avg_draw] * horizon

parameters = {
    "horizon": horizon,
    "elec_costs": prices,
    "load": {"type": "hourly", "value": load_per_hour},
    "control": {
        "max": [max_hp_output] * horizon,
        "min": [min_hp_output] * horizon,
        "units": "Heat Pump Output [kWh]",
        "name": "HPWH Operation",
    },
    "constraints": {
        "storage_capacity": True,
        "max_storage": max_storage,
        "min_storage": min_storage,
        "initial_soc": initial_soc,
        "cheaper_hours": True,
    },
    "hardware": {
        "heatpump": True,
        "COP": [cop] * horizon,
    },
    "price_structure_name": "eTOU-Dyn",
}

# --- Run Easy Shift ---
operation, converged = easy_shift(parameters, verbose=True)

print(f"\nConverged: {converged}")
print(f"\nHourly schedule (kWh):")
for i in range(horizon):
    on_off = "ON " if operation["control"][i] > 0 else "OFF"
    print(f"  Hour {i:2d}: {on_off}  {operation['control'][i]:5.2f} kWh  @ ${prices[i]:.5f}/kWh  cost=${operation['cost'][i]:.5f}")

print(f"\nTotal electricity cost: ${sum(operation['cost']):.5f}")

# --- Plot ---
iteration_plot(operation, parameters)